In [1]:
%load_ext autoreload
%autoreload 2

from bertopic import BERTopic
from urbandev.utils import load_dataset
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired
from transformers import pipeline
from bertopic.representation import TextGeneration

/home/korjakow/.cache/pypoetry/virtualenvs/03-urbandevelopmenttaiwan-HQCD10_a-py3.11/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/korjakow/.cache/pypoetry/virtualenvs/03-urbandevelopmenttaiwan-HQCD10_a-py3.11/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-ji

In [3]:
docs, labels, timestamps = load_dataset(type='sklearn')
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
representation_model_first_stage = KeyBERTInspired()
prompt = "I have a topic described by the following keywords: [KEYWORDS]. Based on the previous keywords, what is this topic about?"
generator = pipeline('text2text-generation', model='google/flan-t5-base')
representation_model_second_stage = TextGeneration(generator)
topic_model = BERTopic(embedding_model='BAAI/bge-large-en', ctfidf_model=ctfidf_model, representation_model=[representation_model_first_stage, representation_model_second_stage], calculate_probabilities=True)
topics, probs = topic_model.fit_transform(docs, y=labels)

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_heatmap()

In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(docs)
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [ ]:
topics_over_time = topic_model.topics_over_time(docs, timestamps, nr_bins=20)

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, topics=topic_model.find_topics("mobility,car, vehicle,streets,city", top_n=10)[0])